In [2]:
import socket
import pickle
import time

def send_message_to_server(socket, client_id):
    while True:
        message = input("Message> ")

        if message.upper() == "EXIT":
            # Enviar mensaje de salida al servidor
            data_to_send = pickle.dumps((message, client_id))
            socket.sendto(data_to_send, server_address)

            # Recibir despedida del servidor
            response, _ = socket.recvfrom(1024)
            print("Message received from Server:", server_address)

            try:
                # Intentar cargar la respuesta como un objeto pickle
                response_data = pickle.loads(response)
                print("Message:", repr(response_data))
            except pickle.UnpicklingError:
                # Si falla, imprimir como cadena binaria
                print("Message:", repr(response))

            break

        # Empaquetar mensaje e identificador del cliente usando pickle
        data_to_send = pickle.dumps((message, client_id))
        socket.sendto(data_to_send, server_address)

        # Esperar respuesta del servidor
        try:
            response, _ = socket.recvfrom(1024)
            response_data = pickle.loads(response)
            print("Message received from Server:", server_address)
            print("Message:", repr(response_data[0]))
            print("Message>" + str(message))

        except socket.timeout:
            print("Waiting for server...")
            time.sleep(5)

# Configurar el socket del cliente
client_id = 1
server_address = ("127.0.0.1", 6780)
socket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
socket.settimeout(5)  # Establecer un tiempo de espera de 5 segundos

try:
    while True:
        try:
            port = int(input("Provide port number (e.g. 6780): "))
            server_address = ("127.0.0.1", port)
            print("Puerto introducido:", port)
            break
        except ValueError:
            print("El puerto debe ser un número")

    # Enviar el ID del cliente al servidor
    data_to_send = pickle.dumps(("ID", client_id))
    socket.sendto(data_to_send, server_address)

    # Esperar confirmación del servidor
    response, _ = socket.recvfrom(1024)
    print(response.decode())

    # Iniciar el envío de mensajes
    send_message_to_server(socket, client_id)

finally:
    # Cerrar el socket al finalizar
    socket.close()


Puerto introducido: 6780
Client 1 connected
Message received from Server: ('127.0.0.1', 6780)
Message: 'Received 4 characters from client 1'
Message>Hola
Message received from Server: ('127.0.0.1', 6780)
Message: 'Received 5 characters from client 1'
Message>Adios
Message received from Server: ('127.0.0.1', 6780)
Message: ('Bye', 3)
